### Imports

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import subjects
from subjects import sns_boxplot_kw, stat_kw
from statannotations.Annotator import Annotator
import seaborn as sns
import pandas as pd

grpdata = subjects.GroupData()
group = ["NSD", "SD"]
ev_pooled = grpdata.ev_in_chunks

### bootstrap testing 

In [ ]:
rng = np.random.default_rng()
subsample_mean = lambda arr: np.mean(rng.choice(arr, size=len(arr), replace=True))

mean_dist_df = []
for g, grp in enumerate(["NSD", "SD"]):
    for t in ev_pooled.zt.unique():
        data = ev_pooled[(ev_pooled.grp == grp) & (ev_pooled.zt == t)].ev_diff.values
        bootstrap_mean = np.array([subsample_mean(data) for _ in range(10000)])
        mean_dist_df.append(pd.DataFrame(dict(zt=t, ev_diff=bootstrap_mean, grp=grp)))

mean_dist_df = pd.concat(mean_dist_df, ignore_index=True)

### Plotting original vs bootstrap

In [ ]:
_,ax = plt.subplots()
plot_kw = dict(
    data=ev_pooled, x="zt", y="ev_diff", hue="grp", hue_order=group, width=0.5, ax=ax
)
sns.violinplot(**plot_kw)
# sns.stripplot(**plot_kw, dodge=True)
ax.tick_params("x", rotation=30)
ax.set_ylabel("Explained variance diff")
ax.set_xlabel("")
ax.legend("", frameon=False)
# stats
orders = ev_pooled.zt.unique()
pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
annotator.configure(test="Mann-Whitney-gt", **stat_kw)

annotator.apply_and_annotate()
annotator.reset_configuration()